In [ ]:
# import
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
import os, inspect
from keras.layers import BatchNormalization
import joblib
tf.random.set_seed(777)

In [ ]:
#현재 접근 폴더 경로 가져오기
NOW =os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) #현재 파일이 위치한 경로

if NOW == '/content': # 구글 드라이브일 경우 접근 허용 및 경로 탐색
  from google.colab import drive
  drive.mount('/content/drive')
  finalPath = os.path.join(NOW,'drive/MyDrive/Colab Notebooks/Final') # 
  NOW = finalPath # 임시

Mounted at /content/drive


### k-fold 데이터와 Labeling된 데이터 가져오기

In [ ]:
#데이터 가져오기

DATAPATH  = os.path.join(NOW,'DATA') #DATA 폴더 접근
Fold =10
for i in range(Fold):
  s1 = "Lold_train_Data_PATH = os.path.join(DATAPATH,'01_split/k_Fold_%d.csv')" %(i+1)
  exec(s1)
  s2 = "Lold_Label_PATH = os.path.join(DATAPATH,'01_split/k_FoldTrainLabel_%d.csv')" %(i+1)
  exec(s2)
  s3 = "k_Fold_%d = pd.read_csv(Lold_train_Data_PATH,sep = ',')" %(i+1)
  exec(s3)
  s4 = "k_Fold_Label_%d = pd.read_csv(Lold_Label_PATH, sep = ',')" %(i+1)
  exec(s4)


### k-fold 교차검증을 위하여 List로 묶기

In [ ]:
#List로 묶기
Fold_list = []
Label_list = []
for i in range(Fold):
  s1="Fold_list.append(k_Fold_%d)" %(i+1)
  exec(s1)
  s2="Label_list.append(k_Fold_Label_%d)" %(i+1)
  exec(s2)

### ann 모델 생성

In [ ]:
def ANN_model(input_data,Initial,NoOfNeuron):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units = NoOfNeuron, activation = 'relu',input_shape =(input_data,)))    # Input  Layer
    model.add(BatchNormalization()) # 배치 정규화
    model.add(keras.layers.Dropout(1/4)) # dropout 
    model.add(keras.layers.Dense(units = NoOfNeuron,                   activation = 'elu'))    # Hidden Layer 1 
    model.add(BatchNormalization()) # 배치 정규화
    model.add(keras.layers.Dense(units = NoOfNeuron,                   activation = 'relu'))    # Hidden Layer 2 
    model.add(BatchNormalization()) # 배치 정규화
    model.add(keras.layers.Dropout(1/2)) # dropout     
    model.add(keras.layers.Dense(units = 5,                            activation = 'softmax')) # Output Layer
    Optimizer=keras.optimizers.Nadam(learning_rate=Initial)
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Optimizer) # 학습률 최적화
    return model

In [ ]:
# 하이퍼 파라미터 정하기
noOfNeuron=512
Size=64
iteration=2000
Patience = 20 # patience를 20으로 설정
learningRate=0.0001
InputDim=int(k_Fold_1.shape[1])

### k-fold을 이용하여 k개 ann모델 생성 

In [ ]:
List_loss=[] # 삭제 예정
List_Accuracy=[] # 삭제 예정
for i in range(Fold):
  X_valid=Fold_list[i]
  y_valid=Label_list[i]
  del Fold_list[i] #valid 데이터 삭제
  del Label_list[i] #valid 데이터 삭제
  X_train = pd.concat(Fold_list)
  y_train = pd.concat(Label_list)
  Fold_list.insert(i,X_valid) # 데이터 복구
  Label_list.insert(i,y_valid) # 데이터 복구
  model= ANN_model(InputDim,learningRate,noOfNeuron)
  early_stopping = EarlyStopping(monitor='val_loss',patience=Patience,restore_best_weights=True,mode='auto') # 과적합 방지하기 위한 함수 생성
  #학습하기
  hist = model.fit(X_train, y_train, epochs=iteration, batch_size=Size,validation_data=(X_valid,y_valid),callbacks=[early_stopping],verbose =0)
  Loss, Accuracy = model.evaluate(X_valid,  y_valid) #삭제예정
  s1="model_%d = model" %(i+1) # 모델 옮기기
  exec(s1)
  List_loss.append(Loss) # 삭제 예정
  s1 ="His_%d = hist" %(i+1) # 삭제 예정
  exec(s1)
  List_Accuracy.append(Accuracy) # 삭제 예정
  print("Fold : ",i+1, "Acuracy :",Accuracy*100)
  s1="weight_%d = model.get_weights()" %(i+1)
  exec(s1)

105/105 [==============================] - 1s 7ms/step - loss: 0.1742 - accuracy: 0.9512
Fold :  1 Acuracy : 95.11904716491699
105/105 [==============================] - 1s 6ms/step - loss: 0.1588 - accuracy: 0.9488
Fold :  2 Acuracy : 94.88095045089722
105/105 [==============================] - 1s 5ms/step - loss: 0.2072 - accuracy: 0.9443
Fold :  3 Acuracy : 94.4345235824585
105/105 [==============================] - 1s 7ms/step - loss: 0.1934 - accuracy: 0.9482
Fold :  4 Acuracy : 94.82142925262451
105/105 [==============================] - 1s 5ms/step - loss: 0.1608 - accuracy: 0.9485
Fold :  5 Acuracy : 94.85118985176086
105/105 [==============================] - 1s 5ms/step - loss: 0.1967 - accuracy: 0.9426
Fold :  6 Acuracy : 94.2559540271759
105/105 [==============================] - 1s 7ms/step - loss: 0.2113 - accuracy: 0.9393
Fold :  7 Acuracy : 93.92856955528259
105/105 [==============================] - 1s 6ms/step - loss: 0.2201 - accuracy: 0.9405
Fold :  8 Acuracy : 94.0

##### weight ann 구하기 feat. 정확도

In [ ]:
Weight_ann =pd.DataFrame(List_Accuracy)

Weight_ann.columns = ['Acc']

### 모델 및 weight 저장하기

In [ ]:
# ann
for i in range(Fold):
  s1="MODELPATH = os.path.join(NOW,'Models/ann_%d.h5')"%(i+1)
  exec(s1)
  s2="model_%d.save(MODELPATH,overwrite=True,include_optimizer=True)" %(i+1)
  exec(s2)

#Weight
Weight_ann_PATH = os.path.join(NOW,'weight/Weight.csv')
Weight_ann.to_csv(Weight_ann_PATH,index=False)